## This practice is based on "Data Analysis with R | R for Data Science and Statistics" by MarinStatsLectures. But I wrote it in Python.

In [1]:
import pandas as pd
from scipy import stats
import numpy as np

np.random.seed(42)

In [2]:
data = pd.read_csv('LungCapData.txt', delimiter='\t')

In [3]:
data.head()

,LungCap,Age,Height,Smoke,Gender,Caesarean
0,6.475,6,62.1,no,male,no
1,10.125,18,74.7,yes,female,no
2,9.550,16,69.7,no,female,yes
3,11.125,14,71.0,no,male,no
4,4.800,5,56.9,no,male,no


In [14]:
print(data['LungCap'].mean())
print(stats.trim_mean(data['LungCap'], 0.5))
print(data['LungCap'].quantile([0.3,0.5,0.8]))
print(np.corrcoef(data['LungCap'], data['Age']))

7.863147586206895
8.0
0.3     6.55
0.5     8.00
0.8    10.20
Name: LungCap, dtype: float64
[[1.        0.8196749]
 [0.8196749 1.       ]]


### Binomial Distribution

In [20]:
k = 3  # Number of successes
n = 20  # Number of trials
p = 1/6  # Probability of success on each trial

print(stats.binom.pmf([0,1,2,3],n,p)) #probability mass function
print(stats.binom.cdf(k,n,p)) #Cumulative Distribution Function

[0.02608405 0.10433621 0.19823881 0.23788657]
0.566545637775669


### Getting the probabilities, percentiles ansd samples from the normal distribution

In [23]:
#cumulative distribution function (CDF) for a normal distribution
x = 70.0  
mu = 75  
sigma = 5  

# Calculate the CDF of x for a normal distribution with mean mu and standard deviation sigma
print(stats.norm.cdf(x, mu, sigma)) #pnorm() in R
print(stats.norm.ppf(0.25, mu, sigma)) #qnorm() in R
print(stats.norm.pdf(70, mu, sigma)) #dnorm() in R

0.15865525393145707
71.62755124901959
0.04839414490382867


In [25]:
#get N sample from normal distribution
n = 40  
mu = 75  
sigma = 5 

samples = np.random.normal(mu,sigma,n)
print(samples)

[80.24965054 78.32270892 68.97786181 78.04841467 71.77936907 59.19296501
 63.40378855 73.36682852 75.82346107 79.68776319 63.90948005 65.26312869
 82.85522352 77.03449281 78.18028281 80.5490668  76.52884299 73.11427328
 76.74926566 74.84200321 77.66181959 66.81087772 67.2515495  70.25184813
 75.89569348 70.02256944 77.62784778 83.30573081 73.57167946 77.84707736
 82.12738097 73.59027836 84.15597717 82.20377793 61.23222429 75.53973456
 73.34493066 77.27067832 73.04423959 83.64624074]


### Finding probabilities and percentiles for t-distribution

In [5]:
#t-Distribution and t-Scores, finding p-value
t=2.3
n=25

print(stats.t.cdf(t,n)) #left-sided 
print(1-stats.t.cdf(t,n)+stats.t.cdf(-t,n)) #two-sided 
 
#find t for 95% confidence
#value of t with 2.5% in each tail
print(stats.t.ppf(0.025, 25))

0.9849632450175315
0.03007350996493699
-2.0595385527532946


### t-test

In [12]:
popmean = 8

res = stats.ttest_1samp(data['LungCap'], popmean, alternative='two-sided')
print(res)
print(res.confidence_interval(0.95))

TtestResult(statistic=-1.3842421466872463, pvalue=0.1667108338400536, df=724)
ConfidenceInterval(low=7.66905224404265, high=8.05724292837114)


### Paired t-test & Wilcoxon

In [8]:
#paired t-test
data = pd.read_csv('BloodPressure.txt', delimiter='\t')

res = stats.ttest_rel(data['Before'], data['After'])
print(res)
print(res.confidence_interval(0.99))

#wilcoxon
res = stats.wilcoxon(data['Before'], data['After'], method='approx', correction=False) #correction - continuity correction; method - method to calculate p-value
print(res)

TtestResult(statistic=3.8882014140500063, pvalue=0.0006985894059870637, df=24)
ConfidenceInterval(low=2.2452785605888215, high=13.754721439411178)
WilcoxonResult(statistic=33.0, pvalue=0.0008220918692306016)


### two-sample t-test for independent groups


In [3]:
res = stats.ttest_ind(data.loc[data['Smoke'] == 'yes','LungCap'], data.loc[data['Smoke'] == 'no','LungCap'], equal_var=False, alternative='two-sided')
print(res)
print(res.confidence_interval(0.95))

TtestResult(statistic=3.649750522823789, pvalue=0.0003927364779843199, df=117.71871064770068)
ConfidenceInterval(low=0.4003547950296715, high=1.3501777526695475)


### Bootstrap Hypothesis Testing

In [29]:
df = pd.read_csv('ChickData.csv')

t_stat1 = abs(df[df['feed']=='casein']['weight'].mean() - df[df['feed']=='meatmeal']['weight'].mean()) #test stat 1
t_stat2 = abs(df[df['feed']=='casein']['weight'].median() - df[df['feed']=='meatmeal']['weight'].median()) #test stat 2

n = df.shape[0]
B = 10000

bs = np.random.choice(a = df['weight'], size = n*B, replace = True).reshape(n,B) #bootstrapping
bs_stat1 = np.zeros(B)
bs_stat2 = np.zeros(B)

for i in range(B): #calculatign test stats for each bootstrap
    bs_stat1[i] = abs(np.mean(bs[:13,i]) - np.mean(bs[13:,i]))
    bs_stat2[i] = abs(np.median(bs[:13,i]) - np.median(bs[13:,i]))

print(np.mean(bs_stat1 >= t_stat1)) #p-value for bootstrap HT t_stat1
print(np.mean(bs_stat2 >= t_stat2)) #p-value for bootstrap HT t_stat2

0.097
0.0691


### Bootstrap Confidence Interval

In [5]:
df = pd.read_csv('ChickData.csv')

t_stat1 = df[df['feed']=='casein']['weight'].mean() - df[df['feed']=='meatmeal']['weight'].mean() #test stat 1
t_stat2 = df[df['feed']=='casein']['weight'].median() - df[df['feed']=='meatmeal']['weight'].median() #test stat 2

n_casein = 12
n_meat = 11
B = 100000

bs_casein = np.random.choice(a = df[df['feed']=='casein']['weight'], size = n_casein*B, replace = True).reshape(n_casein,B)
bs_meat = np.random.choice(a = df[df['feed']=='meatmeal']['weight'], size = n_meat*B, replace = True).reshape(n_meat,B)
bs_stat1 = np.zeros(B)
bs_stat2 = np.zeros(B)

bs_stat1 = np.mean(bs_casein,axis=0) - np.mean(bs_meat,axis=0)
bs_stat2 = np.median(bs_casein,axis=0) - np.median(bs_meat,axis=0)

#95% conf.int
conf_mean = np.quantile(bs_stat1,[0.025,0.975]) 
conf_median = np.quantile(bs_stat2,[0.025,0.975])

print(conf_mean) 
print(conf_median)

[-4.36363636 96.71231061]
[-24.5 116. ]


### Permutation Hypothesis Testing


In [10]:
df = pd.read_csv('ChickData.csv')

t_stat1 = abs(df[df['feed']=='casein']['weight'].mean() - df[df['feed']=='meatmeal']['weight'].mean()) #test stat 1
t_stat2 = abs(df[df['feed']=='casein']['weight'].median() - df[df['feed']=='meatmeal']['weight'].median()) #test stat 2

n = df.shape[0]
P = 10000
bs = np.zeros((n,P))

for i in range(P):
    bs[:,i] = np.random.choice(a = df['weight'], size = n, replace = False) #Permutation

bs_stat1 = np.zeros(P)
bs_stat2 = np.zeros(P)

bs_stat1 = np.abs(np.mean(bs[:13,:],axis=0) - np.mean(bs[13:,:],axis=0))
bs_stat2 = np.abs(np.median(bs[:13,:],axis=0) - np.median(bs[13:,:],axis=0))

print(np.mean(bs_stat1 >= t_stat1)) #p-value for bootstrap HT t_stat1
print(np.mean(bs_stat2 >= t_stat2)) #p-value for bootstrap HT t_stat2

0.1018
0.0529


### ANOVA

In [20]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd


data = pd.read_csv('DietWeigthLoss.txt', delimiter='\t')
model = ols('WeightLoss ~ C(Diet)', data=data).fit()

anova_res = anova_lm(model)
print(anova_res)

            df      sum_sq    mean_sq         F    PR(>F)
C(Diet)    3.0   97.329833  32.443278  6.117526  0.001128
Residual  56.0  296.986667   5.303333       NaN       NaN


In [19]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             WeightLoss   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     6.118
Date:                Thu, 25 Apr 2024   Prob (F-statistic):            0.00113
Time:                        23:38:46   Log-Likelihood:                -133.12
No. Observations:                  60   AIC:                             274.2
Df Residuals:                      56   BIC:                             282.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        9.1800      0.595     15.439      0.000       7.989      10.371
C(Diet)[T.B]    -0.2733      0.841     -0.325      0.746      -1.958       1.411
C(Diet)[T.C]     2.9333      0.841      3.488      0.001       1.249       4.618
C(Diet)[T.D]     1.3600      0.841      1.617      0.111      -0.325       3.045
==============================================================================
Omnibus:                        1.903   Durbin-Watson:                   2.102
Prob(Omnibus):                  0.386   Jarque-Bera (JB):                1.879
Skew:                          -0.391   Prob(JB):                        0.391
Kurtosis:                       2.626   Cond. No.                         4.79
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
#Comparing each pair (TukeyHSD)

tukey = pairwise_tukeyhsd(endog=data['WeightLoss'],     # Data
                          groups=data['Diet'],   # Groups
                          alpha=0.05)               # Significance level

tukey.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
A,B,-0.2733,0.988,-2.4999,1.9533,False
A,C,2.9333,0.0051,0.7067,5.1599,True
A,D,1.36,0.3774,-0.8666,3.5866,False
B,C,3.2067,0.0019,0.9801,5.4333,True
B,D,1.6333,0.2224,-0.5933,3.8599,False
C,D,-1.5733,0.2521,-3.7999,0.6533,False


### Chi Squared

In [13]:
df = pd.read_csv('LungCapData.txt', delimiter='\t')
data = pd.crosstab(df['Gender'], df['Smoke']).to_numpy()

stats.chi2_contingency(data)
stats.fisher_exact(data)

SignificanceResult(statistic=0.7050898203592815, pvalue=0.18454257294554005)

### Simple Linear Regression

In [2]:
import statsmodels.api as sm

data = pd.read_csv('LungCapData.txt', delimiter='\t')
X = data['Age']
Y = data['LungCap']

X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                LungCap   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.671
Method:                 Least Squares   F-statistic:                     1480.
Date:                Mon, 29 Apr 2024   Prob (F-statistic):          4.08e-177
Time:                        23:46:22   Log-Likelihood:                -1334.1
No. Observations:                 725   AIC:                             2672.
Df Residuals:                     723   BIC:                             2681.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1469      0.184      6.249      0.0